In [1]:
import pandas as pd
import polars as pl
import numpy as np
from tqdm import tqdm

In [2]:
def setSeed(x):
    seed_counter = hex(x)[2:]
    seed_counter_binary = bin(x)[2:]

    if len(seed_counter_binary) < cells:
        seed_counter_binary = "0"*(cells-len(seed_counter_binary)) + seed_counter_binary

    for i in range(cells-1, -1, -1):
        row = i // grid_width
        col = i % grid_width
        life_grid[row, col] = (seed_counter_binary[i] == '1')

    state_buffer = np.zeros(shape=(grid_height, grid_width)).astype(int)
    return str.upper(seed_counter)

In [3]:
def evaluateState(x, y):
    for i in range(grid_height):
        for j in range(grid_width):
            counter = 0
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if (di == 0 and dj == 0):
                        continue
                    ni = i + di
                    nj = j + dj
                    if ni >= 0 and ni < grid_height and nj >= 0 and nj < grid_width:
                        counter += x[ni, nj]
            if x[i, j] == 1:
                y[i, j] = int(counter == 2 or counter == 3)
            else:
                y[i, j] = int(counter == 3)
    return y

In [4]:
def evaluateParams(life_grid, state_buffer):
    if np.all(life_grid == 0):
        lifetime = "0"
        period = "N/A"
        period_class = "N/A"
        initial_population = "0"
        return lifetime, period, period_class

    virtual_life_grid = life_grid.copy()
    virtual_state_buffer = state_buffer.copy()

    seen_map = {}
    counter = 0
    seen_map[virtual_life_grid.tobytes()] = counter

    while True:
        evaluateState(virtual_life_grid, virtual_state_buffer)
        counter += 1

        if not np.any(virtual_state_buffer):
            lifetime = str(counter)
            period = "N/A"
            period_class = "N/A"
            break

        state_key = virtual_state_buffer.tobytes()

        if state_key in seen_map:
            lifetime = f"Oscillating from Generation {counter} onwards"
            x = counter - seen_map[state_key]

            if x == 1:
                period_class = "Still Life"
            else:
                period_class = "Oscillator"

            period = str(x)
            break

        seen_map[state_key] = counter
        virtual_life_grid = virtual_state_buffer.copy()

    return lifetime, period, period_class

In [154]:
grid_height = 4
grid_width = 2

cells = grid_width * grid_height
life_grid = np.zeros(shape=(grid_height, grid_width)).astype(int)
state_buffer = np.zeros(shape=(grid_height, grid_width)).astype(int)

df = pd.DataFrame(columns = ["Seed", "Initial Population", "Lifetime", "Period", "Period Class"])

for i in tqdm(range(2**cells)):
    seed = str(grid_height)+"x"+str(grid_width)+"x"+setSeed(i)
    initial_population = life_grid.sum()
    if initial_population > 2:
        lifetime, period, period_class = evaluateParams(life_grid, state_buffer)
        df.loc[len(df)] = [seed, initial_population, lifetime, period, period_class]
    
t = df[~df["Lifetime"].str.contains("Oscillating")].drop(columns=["Period", "Period Class"]).reset_index(drop=True)
o = df[df["Lifetime"].str.contains("Oscillating")].reset_index(drop=True)
del df

if t.shape[0] > 0:
    t["Lifetime"] = t["Lifetime"].astype(int)
    t.to_csv(f"{grid_height}x{grid_width}_t.csv", index=False)

if o.shape[0] > 0:
    o["Period"] = o["Period"].astype(int)
    o.to_csv(f"{grid_height}x{grid_width}_o.csv", index=False)

 ... (more hidden) ...


## Oscillating life forms

In [158]:
o

,Seed,Initial Population,Lifetime,Period,Period Class,Sub Class
0,4x2x7,3,Oscillating from Generation 2 onwards,1,Still Life,Block
1,4x2xB,3,Oscillating from Generation 2 onwards,1,Still Life,Block
2,4x2xD,3,Oscillating from Generation 2 onwards,1,Still Life,Block
3,4x2xE,3,Oscillating from Generation 2 onwards,1,Still Life,Block
4,4x2xF,4,Oscillating from Generation 1 onwards,1,Still Life,Block
...,...,...,...,...,...,...
68,4x2xEB,6,Oscillating from Generation 1 onwards,1,Still Life,Block
69,4x2xED,6,Oscillating from Generation 3 onwards,1,Still Life,Block
70,4x2xEE,6,Oscillating from Generation 3 onwards,1,Still Life,Block
71,4x2xF0,4,Oscillating from Generation 1 onwards,1,Still Life,Block


In [117]:
blocks = ["B", "F", "13", "16", "19", "1A", "1B", "1D", "26", "27",
          "2B", "2E", "32", "34", "35", "36", "4B", "4F", "53", "55",
          "57", "58", "5D", "66", "67", "6B", "73", "8B", "8F", "98",
          "9D", "A6", "A7", "B0", "B5", "C8", "C9", "CA", "CC", "CD",
          "D0", "D1", "D8", "E7", "E8", "E9", "ED", "F1", "F5", "10B",
          "10F", "115", "116", "117", "11E", "126", "127", "12E", "130", "135",
          "14B", "151", "154", "158", "159", "15A", "15E", "166", "16B", "16E",
          "170", "172", "173", "174", "18F", "190", "194", "19C", "19D", "1A0",
          "1A1", "1A2", "1A4", "1A5", "1A8", "1AC", "1AD", "1B0", "1C8", "1C9",
          "1CA", "1CC", "1CE", "1D1", "1D4", "1E0", "1E1", "1E2", "1E3", "1E4"]
tubs = ["5E", "5F", "AA", "E5", "EF", "F4", "133", "137", "14E", "155", 
        "163", "18D", "199", "1AF", "1D9", "1EB", "1EE", "1F4"]
boats = ["6E", "72", "76", "9C", "AB", "AE", "B1", "BB", "BE", "CE", 
         "DC", "E6", "EA", "EC", "FA", "11A", "11B", "12B", "14F", "167",
         "18B", "1A3", "1A9", "1AA", "1B1", "1BA", "1CD", "1E5"]
ships = ["EE", "1AB"]
blinkers = ["38", "92"]

blocks = ["3x3x"+u for u in blocks]
tubs = ["3x3x"+u for u in tubs]
boats = ["3x3x"+u for u in boats]
ships = ["3x3x"+u for u in ships]
blinkers = ["3x3x"+u for u in blinkers]

In [157]:
o["Sub Class"] = "Block"

In [139]:
o.to_csv(f"{grid_height}x{grid_width}_o.csv", index=False)

In [24]:
o["Initial Population"].mean(), o["Initial Population"].median()

(np.float64(4.713333333333333), np.float64(5.0))

In [25]:
o["Period"].mean(), o["Period"].median()

(np.float64(1.0133333333333334), np.float64(1.0))

## Transient life forms

In [151]:
t

,Seed,Initial Population,Lifetime
0,4x1x7,3,2
1,4x1xB,3,1
2,4x1xD,3,1
3,4x1xE,3,2
4,4x1xF,4,2


In [152]:
t["Initial Population"].mean(), t["Initial Population"].median()

(np.float64(3.2), np.float64(3.0))

In [153]:
t["Lifetime"].mean(), t["Lifetime"].median()

(np.float64(1.6), np.float64(2.0))

In [29]:
o.shape[0], t.shape[0], o.shape[0]/t.shape[0]

(150, 316, 0.47468354430379744)

## Analysis
Only states with initial population > 2 have been considered, states with initial population = 1 or 2 are trivially transient. <br>
Various params are as follows: <br>
$\text{ip}_{\text{o}}$ and $\text{ip}_{\text{t}}$ are initial population aggregate(avg/median) for oscillating/transient <br>
$\text{period}$ is the aggregate(avg/median) period of oscillators <br>
$\text{lifetime}$ is the aggregate(avg/median) lifetime of transients <br>
$\text{ot}$ is the ratio of number of oscillating states to number of transient states

### 1x1
Trivial case, it just has a single transient which dies on the next epoch

### 2x1
Also trivial, all transient life forms which die on the next epoch

### 2x2
In a 2x2 grid, with at least 3 initial population, all cells are neighbours to each other and because the population is at least 3, they all survive. Consequently, there are no transients with i.p. > 2. In fact, if the initial population is exactly 3, the 4th cell also comes alive on the next epoch. All oscillators are just blocks which are still life.
$$o = 5$$
$$\text{ip}_{\text{o, avg}} = 3.2$$
$$\text{ip}_{\text{o, median}} = 3.0$$
$$\text{period}_{\text{avg}} = 1.0$$
$$\text{period}_{\text{median}} = 1.0$$
$$\text{blocks} = 5$$
<br>
$$t = 0$$
$$\text{ip}_{\text{t, avg}} = 0.0$$
$$\text{ip}_{\text{t, median}} = 0.0$$
$$\text{lifetime}_{\text{avg}} = \infty$$
$$\text{lifetime}_{\text{median}} = \infty$$
<br>
$$\text{ot} = \infty$$

### 3x1
Trivial, just a single transient exists with ip > 2 and no persistents, the transient is the state where all cells are initially full.
$$o = 0$$
$$\text{ip}_{\text{o, avg}} = 0.0$$
$$\text{ip}_{\text{o, median}} = 0.0$$
$$\text{period}_{\text{avg}} = 0.0$$
$$\text{period}_{\text{median}} = 0.0$$
<br>
$$t = 1$$
$$\text{ip}_{\text{t, avg}} = 3.0$$
$$\text{ip}_{\text{t, median}} = 3.0$$
$$\text{lifetime}_{\text{avg}} = 2.0$$
$$\text{lifetime}_{\text{median}} = 2.0$$
<br>
$$\text{ot} = 0.0$$

### 3x2
The first grid where non-trivial transients and oscillators both exist. All oscillators are 2x2 blocks because there is just not enough space in this grid.
$$o = 18$$
$$\text{ip}_{\text{o, avg}} = 3.55$$
$$\text{ip}_{\text{o, median}} = 4$$
$$\text{period}_{\text{avg}} = 1.0$$
$$\text{period}_{\text{median}} = 1.0$$
$$\text{blocks} = 18$$
<br>
$$t = 24$$
$$\text{ip}_{\text{t, avg}} = 3.83$$
$$\text{ip}_{\text{t, median}} = 3.5$$
$$\text{lifetime}_{\text{avg}} = 2.125$$
$$\text{lifetime}_{\text{median}} = 2.0$$
<br>
$$\text{ot} = 0.75$$

### 3x3
The first grid where a proper oscillator, the blinker appears. The only place it can appear are the middle horizontal section and the middle vertical section. The grid is also just able to accomodate two ships and a good number of boats and tubs. Still, blocks remain the dominating still life due cramped size of the grid favouring the square symmetry.
$$o = 150$$
$$\text{ip}_{\text{o, avg}} = 4.7133$$
$$\text{ip}_{\text{o, median}} = 5.0$$
$$\text{period}_{\text{avg}} = 1.0133$$
$$\text{period}_{\text{median}} = 1.0$$
$$\text{blocks} = 100$$
$$\text{tubs} = 18$$
$$\text{boats} = 28$$
$$\text{ships} = 2$$
$$\text{blinkers} = 2$$
<br>
$$t = 316$$
$$\text{ip}_{\text{t, avg}} = 4.797$$
$$\text{ip}_{\text{t, median}} = 5.0$$
$$\text{lifetime}_{\text{avg}} = 3.019$$
$$\text{lifetime}_{\text{median}} = 3.0$$
<br>
$$\text{ot} = 0.475$$

### 4x1
No oscillators, only weak transients
$$o = 0$$
$$\text{ip}_{\text{o, avg}} = 0.0$$
$$\text{ip}_{\text{o, median}} = 0.0$$
$$\text{period}_{\text{avg}} = 0.0$$
$$\text{period}_{\text{median}} = 0.0$$
<br>
$$t = 5$$
$$\text{ip}_{\text{t, avg}} = 3.2$$
$$\text{ip}_{\text{t, median}} = 3.0$$
$$\text{lifetime}_{\text{avg}} = 1.6$$
$$\text{lifetime}_{\text{median}} = 2.0$$
<br>
$$\text{ot} = 0.0$$